# **Scrapping Data Coment Youtube**

In [ ]:
!pip install selenium --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrap

In [ ]:
data=[]

with Chrome(executable_path=r'C:\webdrivers\chromedriver.exe') as driver:
    wait = WebDriverWait(driver,10)
    driver.get("https://youtu.be/Ov_JKgKJkBg")

    for item in range(10): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\shole\AppData\Local\Temp\ipykernel_8312\4290081225.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\webdrivers\chromedriver.exe') as driver:


In [ ]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df

,comment
0,ID\nSkip navigation\nSign in\nPengamat: Suara ...
1,
2,Saya yakin pemilih Djawa Timur adalah pemilih ...
3,"Indonesia butuh pemimpin cerdas berwawasan, pe..."
4,semoga santiasa dalam lindungan allah SWT pak ...
...,...
181,
182,
183,
184,


## Prepocessing and cleaning

In [ ]:
import re, string
import numpy as np

# Text Cleaning
def cleaning(text):
    # HTML Tag Removal

    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [ ]:
!pip install indonlp


     ---------------------------------------- 0.0/121.9 kB ? eta -:--:--
     ------------------- ------------------- 61.4/121.9 kB 1.1 MB/s eta 0:00:01
     ----------------------------------- -- 112.6/121.9 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ 121.9/121.9 kB 893.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang, emoji_to_words, remove_html])
# steamming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)

In [ ]:
print(df.isna().sum())

comment            0
comment (clean)    8
dtype: int64


In [ ]:
df.dropna(inplace=True)

## Data clean

In [ ]:
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nPengamat: Suara ...,id skip navigation sign ini pengamat suara pdi...
2,Saya yakin pemilih Djawa Timur adalah pemilih ...,saya yakin pemilih djawa timur adalah pemilih ...
3,"Indonesia butuh pemimpin cerdas berwawasan, pe...",indonesia butuh pemimpin cerdas berwawasan pem...
4,semoga santiasa dalam lindungan allah SWT pak ...,semoga santiasa dalam lindungan allah swt pak ...
5,Ganjar Pranowo presiden 2024 sy non politik 🇮🇩,ganjar pranowo presiden saya non politik bende...
...,...,...
174,sebagian besar masyarakat sdh cerdas mana pemi...,sebagian besar masyarakat sudah cerdas mana pe...
175,Merahkan Indonesia Ganjar Pranowo jadi preside...,merahkan indonesia ganjar pranowo jadi preside...
176,SayA jawa timur siap untuk pak Ganjar....!!!,saya jawa timur siap untuk pak ganjar
177,Intinya itu harus jujur harus adil adu gagasan...,intinya itu harus jujur harus adil adu gagasan...


## Export

In [ ]:
# to csv
df.to_csv('datacomment_cnn_fix.csv', index=False)